# Count peptides over all files

In [ ]:
import os
import sys
import logging
from pathlib import Path
import random

import pandas as pd
import ipywidgets as widgets

sys.path.append('/home/jovyan/work/vaep/')
from vaep.io.mq import MaxQuantOutputDynamic

from src.logging import setup_logger_w_file
from src.data_objects import MqAllSummaries
from src.data_objects import PeptideCounter

##################
##### CONFIG #####
##################
from src.config import FOLDER_MQ_TXT_DATA, FOLDER_PROCESSED

from src.config import FOLDER_DATA # project folder for storing the data
print(f"Search Raw-Files on path: {FOLDER_MQ_TXT_DATA}")

##################
### Logging ######
##################

#Delete Jupyter notebook root logger handler
root_logger = logging.getLogger()
root_logger.handlers = []

logger = logging.getLogger('vaep')
logger = setup_logger_w_file(logger, fname_base='log_00_mq_count_peptides')

logger.info('Start with handlers: \n' + "\n".join(f"- {repr(log_)}" for log_ in logger.handlers))

# folders = [folder for folder in  Path(FOLDER_MQ_TXT_DATA).iterdir()]
# w_file = widgets.Dropdown(options=[folder for folder in folders], description='View files')
# w_file

Use samples previously loaded.

In [ ]:
mq_all_summaries = MqAllSummaries()
threshold_ms2_identified = 15_000
folders = mq_all_summaries.get_files_w_min_MS2(threshold=threshold_ms2_identified)
folders[:10]

## Random example

In [ ]:
import random
pd.set_option('max_columns', 60)
random_folder = folders[random.randint(0, len(folders)-1)]
mq_output = MaxQuantOutputDynamic(random_folder)
print(f"peptides.txt from {random_folder!s}")
mq_output.peptides

In [ ]:
use_columns = mq_output.peptides.columns[33:45]
df = mq_output.peptides[use_columns].convert_dtypes() #.to_json('test.json')
df

In [ ]:
df_json_string = df.to_json(orient='index', indent=4)
df_json_string[:1000]

In [ ]:
df_csv = df.to_csv()
df_csv[:1000]

In [ ]:
pd.read_json(df_json_string, orient='index')

In [ ]:
mq_output.peptides.Intensity # as is in peptides.txt, comma seperated thousands

## Count peptides

In [ ]:
peptide_counter = PeptideCounter()
peptide_counter

In [ ]:
try:
    print(peptide_counter.counter.most_common(10),
          len(peptide_counter.loaded),
          sep='\n')
except AttributeError:
    print('New file created.')

In [ ]:
%%time
c = peptide_counter.sum_over_files(folders=folders)

In [ ]:
c.most_common(10) # peptide_counter.counter.most_common(10)

In [ ]:
# To share as python file
N = 1000
with open(f'most_common_{10}_peptides.py', 'w') as f:
    f.write('import pandas as pd\n\n')
    
    #pprint.pformat list -> do this using standardlibrary
    # https://docs.python.org/3/library/pprint.html
    f.write(f"most_common = [\n  ")
    f.write(',\n  '.join(f"{str(t)}" for t in c.most_common(N)))
    f.write("\n]\n\n")
    
    #peptide_counter.loaded()
    
    f.write("pd.DataFrame.from_records(most_common, index='Sequence', columns=['Sequence', 'counts'])\n")

Define missing pattern based on most abundant peptides (hope is to have only few cases in pattern)

## Create peptide intensity dumps for each MQ outputfolder

All folders are stored in a list

Check if the output folder contains already parsed files

maybe this should

In [ ]:
# import json

# import src.config

# with open(src.config.FN_FASTA_DB) as f:
#     data_fasta = json.load(f)
# print(f'Number of proteins in fasta file DB: {len(data_fasta)}')

Some files to investigate in more detail

```
20130408_QE6_LC5_KBS_MNT_QC_HeLa_02  # reversed protein leading razor protein
``` 

In [ ]:
# %%time
# FOLDER_PROCESSED = Path(FOLDER_PROCESSED)
# set_previously_loaded =  {folder.name for folder in FOLDER_PROCESSED.iterdir()}

# FORCE = True

# for folder in folders:
#     if folder.name in set_previously_loaded and not FORCE and (folder / '0_completness_all_genes.json').exists():
#         pass
#     else:
#         logger.info('\n\nProcess: {folder.name}')
#         print(f"Process: {folder.name}")
#         mq_output = MaxQuantOutputDynamic(folder)
#         peptide_extractor = ExtractFromPeptidesTxt(
#             out_folder=FOLDER_PROCESSED, mq_output_object=mq_output, fasta_db=data_fasta)
#         completeness_per_gene = peptide_extractor()

In [ ]:
def load_peptides(folders):
        #logger.info(f'\n\nProcess: {folder.name}')
        #print(f"Process: {folder.name}")
        peptides = pd.read_table(folder / 'peptides.txt' , usecols=[mq.mq_col.SEQUENCE, mq.mq_col.INTENSITY, "Potential contaminant"], index_col=0)
        mask = (peptides[mq.mq_col.INTENSITY] == 0) | (peptides["Potential contaminant"] == '+')
        return {folder.stem: peptides.loc[~mask, mq.mq_col.INTENSITY].to_dict()}

## Theoretial Peptides from used fasta-file

> `01_explore_FASTA.ipynb` (formely `01_FASTA_tryptic_digest.ipynb`)